# Linear Regression with Statsmodels for Movie Revenue

## Activity: Create a Linear Regression Model with Statsmodels for Revenue

- Today we will be working with JUST the data data from the TMDB API for years 2000-2021. 
    - We will prepare the data for modeling
        - Some feature engineering
        - Our usual Preprocessing
        - New steps for statsmodels!
    - We will fit a statsmodels linear regression.
    - We Will inspect the model summary.
    - We will create the visualizations to check assumptions about the residuals.



- Next class we will continue this activity.
    - We will better check all 4 assumptions.
    - We will discuss tactics for dealing with violations of the assumptions. 
    - We will use our coefficients to make stakeholder recommendations.

### Concepts Demonstrated

- [ ] Using `glob` for loading in all final files. 
- [ ] Statsmodels OLS
- [ ] QQ-Plot
- [ ] Residual Plot

# Loading the Data

In [1]:
import json
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector, ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
## fixing random for lesson generation
np.random.seed(321)

In [2]:
pd.set_option('display.max_columns',100)

### 📚 Finding & Loading Batches of Files with `glob`

In [5]:
## Checking what data we already in our Data folder using os.listdir
import os
FOLDER = 'Data/'
file_list = sorted(os.listdir(FOLDER))
file_list

['.DS_Store',
 '2010-2021',
 'combined_tmdb_data.csv.gz',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'final_tmdb_data_2002.csv.gz',
 'final_tmdb_data_2003.csv.gz',
 'final_tmdb_data_2004.csv.gz',
 'final_tmdb_data_2005.csv.gz',
 'final_tmdb_data_2006.csv.gz',
 'final_tmdb_data_2007.csv.gz',
 'final_tmdb_data_2008.csv.gz',
 'final_tmdb_data_2009.csv.gz',
 'title_akas_cleaned.csv.gz',
 'title_basics_cleaned.csv.gz',
 'title_ratings_cleaned.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_api_results_2002.json',
 'tmdb_api_results_2003.json',
 'tmdb_api_results_2004.json',
 'tmdb_api_results_2005.json',
 'tmdb_api_results_2006.json',
 'tmdb_api_results_2007.json',
 'tmdb_api_results_2008.json',
 'tmdb_api_results_2009.json',
 'tmdb_api_results_2010.json',
 'tmdb_api_results_2011.json',
 'tmdb_api_results_2012.json',
 'tmdb_api_results_2013.json',
 'tmdb_api_results_2014.json',
 'tmdb_api_results_2015.json',
 'tmdb_api_results_2016.json',


In [9]:
pd.read_csv(file_list[4], lineterminator='\n')

FileNotFoundError: [Errno 2] No such file or directory: 'final_tmdb_data_2001.csv.gz'

In [10]:
df_list = []
for file in file_list:
    temp_df = pd.read_csv(file, lineterminator='\n')
    df_list.append(temp_df)
len(df_list)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 3131: invalid start byte

In [ ]:
## Try loading in the first .csv.gz file from the list


> Why isn't it working?

In [ ]:
## let's check the filepath 


In [ ]:
## add the folder plus filename


In [ ]:
## try read csv with folder plus filename


- Now we would do that in a loop, and only want to open .csv.gz.
- But there is a better way!
>- Introducing `glob`
    - Glob takes a filepath/query and will find every filename that matches the pattern provided.
    - We use asterisks as wildcards in our query.
    


In [11]:
import glob
## Make a filepath query
q = FOLDER+"final*.csv.gz"
q

'Data/final*.csv.gz'

In [12]:
# Use glob.glob to get COMPLETE filepaths
file_list = glob.glob(q)
file_list

['Data/final_tmdb_data_2006.csv.gz',
 'Data/final_tmdb_data_2008.csv.gz',
 'Data/final_tmdb_data_2004.csv.gz',
 'Data/final_tmdb_data_2000.csv.gz',
 'Data/final_tmdb_data_2002.csv.gz',
 'Data/final_tmdb_data_2007.csv.gz',
 'Data/final_tmdb_data_2009.csv.gz',
 'Data/final_tmdb_data_2005.csv.gz',
 'Data/final_tmdb_data_2001.csv.gz',
 'Data/final_tmdb_data_2003.csv.gz']

In [ ]:
# Use glob.glob to get COMPLETE filepaths


> But where are the rest of the years?

In [ ]:
## in a sub-folder


- Recursive Searching with glob.
    - add a `**/` in the middle of your query to grab any matches from all subfolders. 

In [13]:
# Use glob.glob to get COMPLETE filepaths
q = FOLDER+"**/final*.csv.gz"
file_list = glob.glob(q, recursive=True)
file_list

['Data/final_tmdb_data_2006.csv.gz',
 'Data/final_tmdb_data_2008.csv.gz',
 'Data/final_tmdb_data_2004.csv.gz',
 'Data/final_tmdb_data_2000.csv.gz',
 'Data/final_tmdb_data_2002.csv.gz',
 'Data/final_tmdb_data_2007.csv.gz',
 'Data/final_tmdb_data_2009.csv.gz',
 'Data/final_tmdb_data_2005.csv.gz',
 'Data/final_tmdb_data_2001.csv.gz',
 'Data/final_tmdb_data_2003.csv.gz',
 'Data/2010-2021/final_tmdb_data_2018.csv.gz',
 'Data/2010-2021/final_tmdb_data_2014.csv.gz',
 'Data/2010-2021/final_tmdb_data_2016.csv.gz',
 'Data/2010-2021/final_tmdb_data_2020.csv.gz',
 'Data/2010-2021/final_tmdb_data_2012.csv.gz',
 'Data/2010-2021/final_tmdb_data_2010.csv.gz',
 'Data/2010-2021/final_tmdb_data_2019.csv.gz',
 'Data/2010-2021/final_tmdb_data_2015.csv.gz',
 'Data/2010-2021/final_tmdb_data_2021.csv.gz',
 'Data/2010-2021/final_tmdb_data_2017.csv.gz',
 'Data/2010-2021/final_tmdb_data_2013.csv.gz',
 'Data/2010-2021/final_tmdb_data_2011.csv.gz']

In [14]:
df_list = []
for file in file_list:
    temp_df = pd.read_csv(file, lineterminator='\n')
    df_list.append(temp_df)
len(df_list)

22

In [15]:
# ## use a list comprehension to load in all files into 1 dataframe
df = pd.concat([pd.read_csv(file, lineterminator='\n') for file in file_list])

In [16]:
df

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0144280,0.0,NaN,NaN,100000.0,[],NaN,30356.0,en,Whispers from a Shallow Grave,"When a beautiful photo model disappears, polic...",1.176,/cjRqAh3u2Z0MBx9GtT1k7Z9732w.jpg,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",1997-01-01,0.0,90.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,Whispers from a Shallow Grave,0.0,2.0,2.0,NaN
2,tt0197633,0.0,/1hGrzTajJ5JC6VfPzrUj8Gj5t5k.jpg,NaN,0.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",NaN,58520.0,en,Live Freaky Die Freaky,It is the year 3069 and the Earth has been rob...,3.131,/3QKPZ9SzMcBdqkKdSitQbmRqB2l.jpg,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",2006-01-31,0.0,75.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,Live Freaky Die Freaky,0.0,3.8,8.0,NR
3,tt0204250,0.0,/qKiOZMJH9Riqlk2SkUOgSMZHHJu.jpg,NaN,0.0,"[{'id': 9648, 'name': 'Mystery'}, {'id': 35, '...",NaN,459563.0,en,Death of a Saleswoman,Top-ranking RubberTubber saleswoman Agatha J. ...,0.600,/tIfllOpkhZl5sbqZd1X77iiaJDj.jpg,"[{'id': 90905, 'logo_path': None, 'name': 'Gir...","[{'iso_3166_1': 'US', 'name': 'United States o...",2006-09-18,0.0,77.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,6 housewives. 1 dead body. A trunk full of pla...,Death of a Saleswoman,0.0,3.0,1.0,NaN
4,tt0206634,0.0,/z8FzomPxfCb2nv827VWLSI6oYiB.jpg,NaN,76000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",http://www.universalstudiosentertainment.com/c...,9693.0,en,Children of Men,"In 2027, in a chaotic world in which humans ca...",19.318,/k9IAS4TehZFcKi4HVByxZNPfqex.jpg,"[{'id': 33, 'logo_path': '/8lvHyhjr8oUKOOy2dKX...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",2006-09-22,70595464.0,109.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The future's a thing of the past.,Children of Men,0.0,7.6,5779.0,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2893,tt9282946,0.0,NaN,NaN,0.0,"[{'id': 35, 'name': 'Comedy'}]",NaN,490059.0,ko,도약선생,Won-sik is kicked out by her roommate Woo-jung...,0.914,/rXrDkS3Mpow9q915zrmiH3stAn5.jpg,[],"[{'iso_3166_1': 'KR', 'name': 'South Korea'}]",2011-06-30,0.0,65.0,"[{'english_name': 'Korean', 'iso_639_1': 'ko',...",Released,NaN,Dr. Jump,0.0,7.0,1.0,NaN
2894,tt9385434,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",NaN,566831.0,ja,お米とおっぱい。,"What would you choose, between rice and boobs,...",1.400,/4cmlMKKDkbCxm2Kjs4gfmTFL7XE.jpg,"[{'id': 11725, 'logo_path': '/3OZxd70DZ1LbVelm...","[{'iso_3166_1': 'JP', 'name': 'Japan'}]",2011-07-01,0.0,102.0,"[{'english_name': 'Japanese', 'iso_639_1': 'ja...",Released,NaN,Rice and Boobs,0.0,0.0,0.0,NaN
2895,tt9452878,0.0,NaN,NaN,0.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,108925.0,en,The Wrong Ferarri,The Wrong Ferarri is a feature-film written an...,1.884,NaN,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",2011-04-04,0.0,72.0,[],Released,NaN,The Wrong Ferarri,0.0,2.0,1.0,NaN
2896,tt9519786,0.0,/oof2qSqrH1PAe9yEaBnId1P326G.jpg,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",NaN,874426.0,zh,North point,NaN,1.343,/ynnOmcXSIQlWnCykRWA3o0rPsAv.jpg,"[{'id': 158558, 'logo_path': None, 'name': '上海...",[],2011-11-04,0.0,0.0,[],Released,NaN,North point,0.0,0.0,0.0,NaN


- Dealing with ParserErrors with "possibly malformed files"

    - for a reason I do not fully understand yet, some of the files I downloaded error if I try to read them.
        - "ParserError: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.`
    - After some googling, the fix was to add `lineterminator='\n'` to pd.read_csv


In [ ]:
## use a list comprehension to load in all files into 1 dataframe


In [17]:
# remove ids that are 0  and then reset index
df = df.loc[ df['imdb_id']!='0']
df

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
1,tt0144280,0.0,NaN,NaN,100000.0,[],NaN,30356.0,en,Whispers from a Shallow Grave,"When a beautiful photo model disappears, polic...",1.176,/cjRqAh3u2Z0MBx9GtT1k7Z9732w.jpg,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",1997-01-01,0.0,90.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,Whispers from a Shallow Grave,0.0,2.0,2.0,NaN
2,tt0197633,0.0,/1hGrzTajJ5JC6VfPzrUj8Gj5t5k.jpg,NaN,0.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",NaN,58520.0,en,Live Freaky Die Freaky,It is the year 3069 and the Earth has been rob...,3.131,/3QKPZ9SzMcBdqkKdSitQbmRqB2l.jpg,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",2006-01-31,0.0,75.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,Live Freaky Die Freaky,0.0,3.8,8.0,NR
3,tt0204250,0.0,/qKiOZMJH9Riqlk2SkUOgSMZHHJu.jpg,NaN,0.0,"[{'id': 9648, 'name': 'Mystery'}, {'id': 35, '...",NaN,459563.0,en,Death of a Saleswoman,Top-ranking RubberTubber saleswoman Agatha J. ...,0.600,/tIfllOpkhZl5sbqZd1X77iiaJDj.jpg,"[{'id': 90905, 'logo_path': None, 'name': 'Gir...","[{'iso_3166_1': 'US', 'name': 'United States o...",2006-09-18,0.0,77.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,6 housewives. 1 dead body. A trunk full of pla...,Death of a Saleswoman,0.0,3.0,1.0,NaN
4,tt0206634,0.0,/z8FzomPxfCb2nv827VWLSI6oYiB.jpg,NaN,76000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",http://www.universalstudiosentertainment.com/c...,9693.0,en,Children of Men,"In 2027, in a chaotic world in which humans ca...",19.318,/k9IAS4TehZFcKi4HVByxZNPfqex.jpg,"[{'id': 33, 'logo_path': '/8lvHyhjr8oUKOOy2dKX...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",2006-09-22,70595464.0,109.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The future's a thing of the past.,Children of Men,0.0,7.6,5779.0,R
5,tt0244521,0.0,/4W53mm2nvOtiOuPNW2oiBm9pmUZ.jpg,NaN,0.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 80, 'nam...",NaN,9925.0,en,Funny Money,"Henry Perkins, a mild-mannered accountant, acc...",5.591,/oG3jWUZiDqqrdk3oKwPEkuwoN89.jpg,"[{'id': 110622, 'logo_path': None, 'name': 'FW...","[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'is...",2006-01-01,0.0,98.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Henry Perkins accidentally trades briefcases w...,Funny Money,0.0,5.4,39.0,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2893,tt9282946,0.0,NaN,NaN,0.0,"[{'id': 35, 'name': 'Comedy'}]",NaN,490059.0,ko,도약선생,Won-sik is kicked out by her roommate Woo-jung...,0.914,/rXrDkS3Mpow9q915zrmiH3stAn5.jpg,[],"[{'iso_3166_1': 'KR', 'name': 'South Korea'}]",2011-06-30,0.0,65.0,"[{'english_name': 'Korean', 'iso_639_1': 'ko',...",Released,NaN,Dr. Jump,0.0,7.0,1.0,NaN
2894,tt9385434,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",NaN,566831.0,ja,お米とおっぱい。,"What would you choose, between rice and boobs,...",1.400,/4cmlMKKDkbCxm2Kjs4gfmTFL7XE.jpg,"[{'id': 11725, 'logo_path': '/3OZxd70DZ1LbVelm...","[{'iso_3166_1': 'JP', 'name': 'Japan'}]",2011-07-01,0.0,102.0,"[{'english_name': 'Japanese', 'iso_639_1': 'ja...",Released,NaN,Rice and Boobs,0.0,0.0,0.0,NaN
2895,tt9452878,0.0,NaN,NaN,0.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,108925.0,en,The Wrong Ferarri,The Wrong Ferarri is a feature-film written an...,1.884,NaN,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",2011-04-04,0.0,72.0,[],Released,NaN,The Wrong Ferarri,0.0,2.0,1.0,NaN
2896,tt9519786,0.0,/oof2qSqrH1PAe9yEaBnId1P326G.jpg,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",NaN,874426.0,zh,North point,NaN,1.343,/ynnOmcXSIQlWnCykRWA3o0rPsAv.jpg,"[{'id': 158558, 'logo_path': None, 'name': '上海...",[],2011-11-04,0.0,0.0,[],Released,NaN,Nort

In [18]:

df.reset_index(inplace=True)


In [19]:
## saving the combined csv to disk
df.to_csv('Data/combined_all_data.csv.gz', compression="gzip", index = False)

# Preprocessing

In [20]:
## Columns to exclude
drop_cols = ['backdrop_path','backdrop_path','original_title','overview',
                 'poster_path','status','tagline','id','homepage',
                 'production_countries','video','production_companies','spoken_languages',
            'original_language']

## drop columns
df = df.drop(columns=drop_cols)
df

,index,imdb_id,adult,belongs_to_collection,budget,genres,popularity,release_date,revenue,runtime,title,vote_average,vote_count,certification
0,1,tt0144280,0.0,NaN,100000.0,[],1.176,1997-01-01,0.0,90.0,Whispers from a Shallow Grave,2.0,2.0,NaN
1,2,tt0197633,0.0,NaN,0.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",3.131,2006-01-31,0.0,75.0,Live Freaky Die Freaky,3.8,8.0,NR
2,3,tt0204250,0.0,NaN,0.0,"[{'id': 9648, 'name': 'Mystery'}, {'id': 35, '...",0.600,2006-09-18,0.0,77.0,Death of a Saleswoman,3.0,1.0,NaN
3,4,tt0206634,0.0,NaN,76000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",19.318,2006-09-22,70595464.0,109.0,Children of Men,7.6,5779.0,R
4,5,tt0244521,0.0,NaN,0.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 80, 'nam...",5.591,2006-01-01,0.0,98.0,Funny Money,5.4,39.0,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60125,2893,tt9282946,0.0,NaN,0.0,"[{'id': 35, 'name': 'Comedy'}]",0.914,2011-06-30,0.0,65.0,Dr. Jump,7.0,1.0,NaN
60126,2894,tt9385434,0.0,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",1.400,2011-07-01,0.0,102.0,Rice and Boobs,0.0,0.0,NaN
60127,2895,tt9452878,0.0,NaN,0.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1.884,2011-04-04,0.0,72.0,The Wrong Ferarri,2.0,1.0,NaN
60128,2896,tt9519786,0.0,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",1.343,2011-11-04,0.0,0.0,North point,0.0,0.0,NaN


## Feature Engineering


- belongs to collection: convert to boolean
- Genres: get just name and manually OHE
- Cleaning Categories in Certification
- Converting release date to year, month, and day.

### belongs to collection

In [21]:

df['belongs_to_collection'].value_counts()

{'id': 39199, 'name': 'Detective Conan Collection', 'poster_path': '/1wBfr532NOQK68wlo5ApjCmiQIB.jpg', 'backdrop_path': '/9bogrpii4e61SR6a9qLHow7I46U.jpg'}               18
{'id': 148065, 'name': 'Doraemon Collection', 'poster_path': '/4TLSP1KD1uAlp2q1rTrc6SFlktX.jpg', 'backdrop_path': '/rc6OFcSasL5YxBRPUQVwxmVF6h5.jpg'}                     16
{'id': 403643, 'name': 'Troublesome Night Collection', 'poster_path': '/bPTx3TP4UJTHQfcLx4qIub9LXmi.jpg', 'backdrop_path': '/n3a7zF5GuxM2X8oPF6pKXqYS6ER.jpg'}            15
{'id': 23456, 'name': 'One Piece Collection', 'poster_path': '/nvAPotUDNcKStSOv2ojGZBNOX8A.jpg', 'backdrop_path': '/3RqSKjokWlXyTBUt3tcR9CrOG57.jpg'}                     13
{'id': 23616, 'name': 'Naruto Collection', 'poster_path': '/q9rrfRgPUFkFqDF74jlvNYp3RpN.jpg', 'backdrop_path': '/prLI2SNNkd9wcQkFh9iWXzQtR5D.jpg'}                        11
                                                                                                                                       

In [22]:
# there are 3,700+ movies that belong to collections
df['belongs_to_collection'] = df['belongs_to_collection'].isna()
df['belongs_to_collection'].value_counts(dropna=False)

True     56392
False     3738
Name: belongs_to_collection, dtype: int64

In [ ]:
## Use .notna() to get True if it belongs to a collection


### genre

In [23]:

df.loc[5,'genres']

"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name': 'Crime'}, {'id': 53, 'name': 'Thriller'}]"

In [24]:
## Function to get just the genre names as a list 
import json
def get_genre_name(x):
    x = x.replace("'",'"')
    x = json.loads(x)
    
    genres = []
    for genre in x:
        genres.append(genre['name'])
    return genres

In [25]:
## Use our function and exploding the new column
df['genre_list'] = df['genres'].apply(get_genre_name)
df.head()

,index,imdb_id,adult,belongs_to_collection,budget,genres,popularity,release_date,revenue,runtime,title,vote_average,vote_count,certification,genre_list
0,1,tt0144280,0.0,True,100000.0,[],1.176,1997-01-01,0.0,90.0,Whispers from a Shallow Grave,2.0,2.0,NaN,[]
1,2,tt0197633,0.0,True,0.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",3.131,2006-01-31,0.0,75.0,Live Freaky Die Freaky,3.8,8.0,NR,"[Animation, Comedy, Music, Science Fiction]"
2,3,tt0204250,0.0,True,0.0,"[{'id': 9648, 'name': 'Mystery'}, {'id': 35, '...",0.600,2006-09-18,0.0,77.0,Death of a Saleswoman,3.0,1.0,NaN,"[Mystery, Comedy]"
3,4,tt0206634,0.0,True,76000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",19.318,2006-09-22,70595464.0,109.0,Children of Men,7.6,5779.0,R,"[Drama, Action, Thriller, Science Fiction]"
4,5,tt0244521,0.0,True,0.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 80, 'nam...",5.591,2006-01-01,0.0,98.0,Funny Money,5.4,39.0,R,"[Comedy, Crime]"


In [26]:
df_explode = df.explode('genre_list')
df_explode

,index,imdb_id,adult,belongs_to_collection,budget,genres,popularity,release_date,revenue,runtime,title,vote_average,vote_count,certification,genre_list
0,1,tt0144280,0.0,True,100000.0,[],1.176,1997-01-01,0.0,90.0,Whispers from a Shallow Grave,2.0,2.0,NaN,NaN
1,2,tt0197633,0.0,True,0.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",3.131,2006-01-31,0.0,75.0,Live Freaky Die Freaky,3.8,8.0,NR,Animation
1,2,tt0197633,0.0,True,0.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",3.131,2006-01-31,0.0,75.0,Live Freaky Die Freaky,3.8,8.0,NR,Comedy
1,2,tt0197633,0.0,True,0.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",3.131,2006-01-31,0.0,75.0,Live Freaky Die Freaky,3.8,8.0,NR,Music
1,2,tt0197633,0.0,True,0.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",3.131,2006-01-31,0.0,75.0,Live Freaky Die Freaky,3.8,8.0,NR,Science Fiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60127,2895,tt9452878,0.0,True,0.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1.884,2011-04-04,0.0,72.0,The Wrong Ferarri,2.0,1.0,NaN,Drama
60127,2895,tt9452878,0.0,True,0.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1.884,2011-04-04,0.0,72.0,The Wrong Ferarri,2.0,1.0,NaN,Music
60128,2896,tt9519786,0.0,True,0.0,"[{'id': 18, 'name': 'Drama'}]",1.343,2011-11-04,0.0,0.0,North point,0.0,0.0,NaN,Drama
60129,2897,tt9547900,0.0,True,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",1.343,2011-05-13,0.0,0.0,百年情书,0.0,0.0,NaN,Drama


In [27]:
## save unique genres
unique_genres = df_explode['genre_list'].dropna().unique()
unique_genres

array(['Animation', 'Comedy', 'Music', 'Science Fiction', 'Mystery',
       'Drama', 'Action', 'Thriller', 'Crime', 'Romance', 'Horror',
       'Adventure', 'Family', 'Fantasy', 'History', 'Documentary', 'War',
       'Western', 'TV Movie'], dtype=object)

In [28]:
## Manually One-Hot-Encode Genres
for genre in unique_genres:
    df[f"Genre_{genre}"] = df['genres'].str.contains(genre,regex=False)    
df

,index,imdb_id,adult,belongs_to_collection,budget,genres,popularity,release_date,revenue,runtime,title,vote_average,vote_count,certification,genre_list,Genre_Animation,Genre_Comedy,Genre_Music,Genre_Science Fiction,Genre_Mystery,Genre_Drama,Genre_Action,Genre_Thriller,Genre_Crime,Genre_Romance,Genre_Horror,Genre_Adventure,Genre_Family,Genre_Fantasy,Genre_History,Genre_Documentary,Genre_War,Genre_Western,Genre_TV Movie
0,1,tt0144280,0.0,True,100000.0,[],1.176,1997-01-01,0.0,90.0,Whispers from a Shallow Grave,2.0,2.0,NaN,[],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,2,tt0197633,0.0,True,0.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",3.131,2006-01-31,0.0,75.0,Live Freaky Die Freaky,3.8,8.0,NR,"[Animation, Comedy, Music, Science Fiction]",True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,3,tt0204250,0.0,True,0.0,"[{'id': 9648, 'name': 'Mystery'}, {'id': 35, '...",0.600,2006-09-18,0.0,77.0,Death of a Saleswoman,3.0,1.0,NaN,"[Mystery, Comedy]",False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,4,tt0206634,0.0,True,76000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",19.318,2006-09-22,70595464.0,109.0,Children of Men,7.6,5779.0,R,"[Drama, Action, Thriller, Science Fiction]",False,False,False,True,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False
4,5,tt0244521,0.0,True,0.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 80, 'nam...",5.591,2006-01-01,0.0,98.0,Funny Money,5.4,39.0,R,"[Comedy, Crime]",False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60125,2893,tt9282946,0.0,True,0.0,"[{'id': 35, 'name': 'Comedy'}]",0.914,2011-06-30,0.0,65.0,Dr. Jump,7.0,1.0,NaN,[Comedy],False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
60126,2894,tt9385434,0.0,True,0.0,"[{'id': 18, 'name': 'Drama'}]",1.400,2011-07-01,0.0,102.0,Rice and Boobs,0.0,0.0,NaN,[Drama],False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
60127,2895,tt9452878,0.0,True,0.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1.884,2011-04-04,0.0,72.0,The Wrong Ferarri,2.0,1.0,NaN,"[Comedy, Drama, Music]",False,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
60128,2896,tt9519786,0.0,True,0.0,"[{'id': 18, 'name': 'Drama'}]",1.343,2011-11-04,0.0,0.0,North point,0.0,0.0,NaN,[Drama],False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False


In [29]:
## Drop original genre cols
df = df.drop(columns=['genres','genre_list','index'])
df

,imdb_id,adult,belongs_to_collection,budget,popularity,release_date,revenue,runtime,title,vote_average,vote_count,certification,Genre_Animation,Genre_Comedy,Genre_Music,Genre_Science Fiction,Genre_Mystery,Genre_Drama,Genre_Action,Genre_Thriller,Genre_Crime,Genre_Romance,Genre_Horror,Genre_Adventure,Genre_Family,Genre_Fantasy,Genre_History,Genre_Documentary,Genre_War,Genre_Western,Genre_TV Movie
0,tt0144280,0.0,True,100000.0,1.176,1997-01-01,0.0,90.0,Whispers from a Shallow Grave,2.0,2.0,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,tt0197633,0.0,True,0.0,3.131,2006-01-31,0.0,75.0,Live Freaky Die Freaky,3.8,8.0,NR,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,tt0204250,0.0,True,0.0,0.600,2006-09-18,0.0,77.0,Death of a Saleswoman,3.0,1.0,NaN,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,tt0206634,0.0,True,76000000.0,19.318,2006-09-22,70595464.0,109.0,Children of Men,7.6,5779.0,R,False,False,False,True,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False
4,tt0244521,0.0,True,0.0,5.591,2006-01-01,0.0,98.0,Funny Money,5.4,39.0,R,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60125,tt9282946,0.0,True,0.0,0.914,2011-06-30,0.0,65.0,Dr. Jump,7.0,1.0,NaN,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
60126,tt9385434,0.0,True,0.0,1.400,2011-07-01,0.0,102.0,Rice and Boobs,0.0,0.0,NaN,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
60127,tt9452878,0.0,True,0.0,1.884,2011-04-04,0.0,72.0,The Wrong Ferarri,2.0,1.0,NaN,False,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
60128,tt9519786,0.0,True,0.0,1.343,2011-11-04,0.0,0.0,North point,0.0,0.0,NaN,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False


### certification

In [30]:
## Checking Certification values
df['certification'].value_counts(dropna=False)

NaN                                45506
R                                   6097
NR                                  3261
PG-13                               3224
PG                                  1432
G                                    442
NC-17                                156
Unrated                                5
ScreamFest Horror Film Festival        1
UR                                     1
Not Rated                              1
-                                      1
PG-13                                  1
10                                     1
R                                      1
Name: certification, dtype: int64

In [31]:
# fix extra space certs
df['certification'] = df['certification'].str.strip()
df['certification'].value_counts(dropna=False)

NaN                                45506
R                                   6098
NR                                  3261
PG-13                               3225
PG                                  1432
G                                    442
NC-17                                156
Unrated                                5
ScreamFest Horror Film Festival        1
UR                                     1
Not Rated                              1
-                                      1
10                                     1
Name: certification, dtype: int64

In [32]:
## fix certification col
repl_cert = {'UR':'NR',
             'Not Rated':'NR',
             'Unrated':'NR',
             '-':'NR',
             '10':np.nan,
             'ScreamFest Horror Film Festival':'NR'}
df['certification'] = df['certification'].replace(repl_cert)#

df['certification'].value_counts()

R        6098
NR       3270
PG-13    3225
PG       1432
G         442
NC-17     156
Name: certification, dtype: int64

### Converting year to sep features

In [33]:
## split release date into 3 columns
new_cols = ['year','month','day']
df[new_cols] = df['release_date'].str.split('-',expand=True)
df[new_cols] = df[new_cols].astype(float)
df.head()

,imdb_id,adult,belongs_to_collection,budget,popularity,release_date,revenue,runtime,title,vote_average,vote_count,certification,Genre_Animation,Genre_Comedy,Genre_Music,Genre_Science Fiction,Genre_Mystery,Genre_Drama,Genre_Action,Genre_Thriller,Genre_Crime,Genre_Romance,Genre_Horror,Genre_Adventure,Genre_Family,Genre_Fantasy,Genre_History,Genre_Documentary,Genre_War,Genre_Western,Genre_TV Movie,year,month,day
0,tt0144280,0.0,True,100000.0,1.176,1997-01-01,0.0,90.0,Whispers from a Shallow Grave,2.0,2.0,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,1997.0,1.0,1.0
1,tt0197633,0.0,True,0.0,3.131,2006-01-31,0.0,75.0,Live Freaky Die Freaky,3.8,8.0,NR,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,2006.0,1.0,31.0
2,tt0204250,0.0,True,0.0,0.600,2006-09-18,0.0,77.0,Death of a Saleswoman,3.0,1.0,NaN,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,2006.0,9.0,18.0
3,tt0206634,0.0,True,76000000.0,19.318,2006-09-22,70595464.0,109.0,Children of Men,7.6,5779.0,R,False,False,False,True,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False,2006.0,9.0,22.0
4,tt0244521,0.0,True,0.0,5.591,2006-01-01,0.0,98.0,Funny Money,5.4,39.0,R,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,2006.0,1.0,1.0


In [35]:
## drop original feature
df = df.drop(columns=['release_date'])

KeyError: "['release_date'] not found in axis"

## Train Test Split

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60130 entries, 0 to 60129
Data columns (total 33 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_id                60130 non-null  object 
 1   adult                  60130 non-null  float64
 2   belongs_to_collection  60130 non-null  bool   
 3   budget                 60130 non-null  float64
 4   popularity             60130 non-null  float64
 5   revenue                60130 non-null  float64
 6   runtime                59298 non-null  float64
 7   title                  60130 non-null  object 
 8   vote_average           60130 non-null  float64
 9   vote_count             60130 non-null  float64
 10  certification          14623 non-null  object 
 11  Genre_Animation        60130 non-null  bool   
 12  Genre_Comedy           60130 non-null  bool   
 13  Genre_Music            60130 non-null  bool   
 14  Genre_Science Fiction  60130 non-null  bool   
 15  Ge

In [37]:
## Make x and y variables
drop_for_model = ['title','imdb_id']

y = df['revenue'].copy()
X = df.drop(columns=['revenue',*drop_for_model]).copy()

X_train, X_test, y_train, y_test = train_test_split(X,y)#, random_state=321)
X_train.head()

,adult,belongs_to_collection,budget,popularity,runtime,vote_average,vote_count,certification,Genre_Animation,Genre_Comedy,Genre_Music,Genre_Science Fiction,Genre_Mystery,Genre_Drama,Genre_Action,Genre_Thriller,Genre_Crime,Genre_Romance,Genre_Horror,Genre_Adventure,Genre_Family,Genre_Fantasy,Genre_History,Genre_Documentary,Genre_War,Genre_Western,Genre_TV Movie,year,month,day
9538,0.0,True,0.0,0.600,70.0,0.0,0.0,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,2007.0,10.0,24.0
16553,0.0,True,0.0,9.087,100.0,6.1,141.0,NaN,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,2018.0,12.0,7.0
34106,0.0,True,10.0,1.842,89.0,4.5,4.0,NaN,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,2012.0,3.0,18.0
3804,0.0,True,0.0,1.092,97.0,0.0,0.0,NaN,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,2008.0,8.0,7.0
37747,0.0,True,0.0,1.209,98.0,2.8,8.0,NaN,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,2019.0,7.0,12.0


In [38]:
X_train.isna().sum()

adult                        0
belongs_to_collection        0
budget                       0
popularity                   0
runtime                    626
vote_average                 0
vote_count                   0
certification            34093
Genre_Animation              0
Genre_Comedy                 0
Genre_Music                  0
Genre_Science Fiction        0
Genre_Mystery                0
Genre_Drama                  0
Genre_Action                 0
Genre_Thriller               0
Genre_Crime                  0
Genre_Romance                0
Genre_Horror                 0
Genre_Adventure              0
Genre_Family                 0
Genre_Fantasy                0
Genre_History                0
Genre_Documentary            0
Genre_War                    0
Genre_Western                0
Genre_TV Movie               0
year                       760
month                      760
day                        760
dtype: int64

In [39]:
## make cat selector and using it to save list of column names
cat_select = make_column_selector(dtype_include='object')
cat_cols = cat_select(X_train)
cat_cols

['certification']

In [40]:
## select manually OHE cols for later
bool_select = make_column_selector(dtype_include='bool')
already_ohe_cols = bool_select(X_train)
already_ohe_cols

['belongs_to_collection',
 'Genre_Animation',
 'Genre_Comedy',
 'Genre_Music',
 'Genre_Science Fiction',
 'Genre_Mystery',
 'Genre_Drama',
 'Genre_Action',
 'Genre_Thriller',
 'Genre_Crime',
 'Genre_Romance',
 'Genre_Horror',
 'Genre_Adventure',
 'Genre_Family',
 'Genre_Fantasy',
 'Genre_History',
 'Genre_Documentary',
 'Genre_War',
 'Genre_Western',
 'Genre_TV Movie']

In [41]:
## convert manual ohe to int
X_train[already_ohe_cols] = X_train[already_ohe_cols].astype(int)
X_test[already_ohe_cols] = X_test[already_ohe_cols].astype(int)

In [42]:
## make num selector and using it to save list of column names
num_select = make_column_selector(dtype_include='number')
num_cols = num_select(X_train)
num_cols

['adult',
 'belongs_to_collection',
 'budget',
 'popularity',
 'runtime',
 'vote_average',
 'vote_count',
 'Genre_Animation',
 'Genre_Comedy',
 'Genre_Music',
 'Genre_Science Fiction',
 'Genre_Mystery',
 'Genre_Drama',
 'Genre_Action',
 'Genre_Thriller',
 'Genre_Crime',
 'Genre_Romance',
 'Genre_Horror',
 'Genre_Adventure',
 'Genre_Family',
 'Genre_Fantasy',
 'Genre_History',
 'Genre_Documentary',
 'Genre_War',
 'Genre_Western',
 'Genre_TV Movie',
 'year',
 'month',
 'day']

In [43]:
## make pipelines
cat_pipe = make_pipeline(SimpleImputer(strategy='constant',
                                       fill_value='MISSING'),
                         OneHotEncoder(handle_unknown='ignore', sparse=False))
num_pipe = make_pipeline(SimpleImputer(strategy='mean'),#StandardScaler()
                        )

preprocessor = make_column_transformer((cat_pipe,cat_cols),
                                        (num_pipe, num_cols),
                                       ('passthrough',already_ohe_cols))# remainder='passthrough')
preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='MISSING',
                                                                strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 ['certification']),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer())]),
                                 ['adult', 'belongs_to_collection', 'budget',
                                  'popularity', 'r...
                                ('passthrough', 'passthrough',
                                 ['belongs_to_collection', 'Genre_Animation',
                                  'Genre_Comedy', 'Genre_Music',
                                  'Genre_Science Fiction', 'Genre_Mystery',
                                  'Genre_Drama', 'Genre_Action',
                                  'Genre_Thriller', 'Genre_Crime',
                                  'Genre_Romance', 'Genre_Horror',
                                  'Genre_Adventure', 'Genre_Family',
                                  'Genre_Fantasy', 'Genre_History',
                                  'Genre_Documentary', 'Genre_War',
                                  'Genre_Western', 'Genre_TV Movie'])])

In [44]:
## fit the col transformer
preprocessor.fit(X_train)

## Finding the categorical pipeline in our col transformer.
preprocessor.named_transformers_['pipeline-1']

Pipeline(steps=[('simpleimputer',
                 SimpleImputer(fill_value='MISSING', strategy='constant')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

In [45]:
## B) Using list-slicing to find the encoder 
cat_features = preprocessor.named_transformers_['pipeline-1'][-1].get_feature_names_out(cat_cols)

## Create the empty list
final_features = [*cat_features,*num_cols,*already_ohe_cols]
len(final_features)

56

In [46]:
## checking shape matches len final features
preprocessor.transform(X_train).shape

(45097, 56)

In [47]:
## make X_train_tf 
X_train_tf = pd.DataFrame( preprocessor.transform(X_train), 
                          columns=final_features, index=X_train.index)
X_train_tf.head()

,certification_G,certification_MISSING,certification_NC-17,certification_NR,certification_PG,certification_PG-13,certification_R,adult,belongs_to_collection,budget,popularity,runtime,vote_average,vote_count,Genre_Animation,Genre_Comedy,Genre_Music,Genre_Science Fiction,Genre_Mystery,Genre_Drama,Genre_Action,Genre_Thriller,Genre_Crime,Genre_Romance,Genre_Horror,Genre_Adventure,Genre_Family,Genre_Fantasy,Genre_History,Genre_Documentary,Genre_War,Genre_Western,Genre_TV Movie,year,month,day,belongs_to_collection,Genre_Animation,Genre_Comedy,Genre_Music,Genre_Science Fiction,Genre_Mystery,Genre_Drama,Genre_Action,Genre_Thriller,Genre_Crime,Genre_Romance,Genre_Horror,Genre_Adventure,Genre_Family,Genre_Fantasy,Genre_History,Genre_Documentary,Genre_War,Genre_Western,Genre_TV Movie
9538,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.600,70.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2007.0,10.0,24.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16553,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.087,100.0,6.1,141.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018.0,12.0,7.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34106,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,10.0,1.842,89.0,4.5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2012.0,3.0,18.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3804,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.092,97.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2008.0,8.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37747,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.209,98.0,2.8,8.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019.0,7.0,12.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
## make X_test_tf 

X_test_tf = pd.DataFrame( preprocessor.transform(X_test), 
                         columns=final_features, index=X_test.index)
X_test_tf.head()

,certification_G,certification_MISSING,certification_NC-17,certification_NR,certification_PG,certification_PG-13,certification_R,adult,belongs_to_collection,budget,popularity,runtime,vote_average,vote_count,Genre_Animation,Genre_Comedy,Genre_Music,Genre_Science Fiction,Genre_Mystery,Genre_Drama,Genre_Action,Genre_Thriller,Genre_Crime,Genre_Romance,Genre_Horror,Genre_Adventure,Genre_Family,Genre_Fantasy,Genre_History,Genre_Documentary,Genre_War,Genre_Western,Genre_TV Movie,year,month,day,belongs_to_collection,Genre_Animation,Genre_Comedy,Genre_Music,Genre_Science Fiction,Genre_Mystery,Genre_Drama,Genre_Action,Genre_Thriller,Genre_Crime,Genre_Romance,Genre_Horror,Genre_Adventure,Genre_Family,Genre_Fantasy,Genre_History,Genre_Documentary,Genre_War,Genre_Western,Genre_TV Movie
19252,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.911,90.0,4.5,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018.0,2.0,16.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40663,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.911,148.0,5.1,19.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019.0,1.0,12.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35595,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.728,78.0,5.4,4.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2010.0,9.0,10.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
979,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.719,97.0,6.1,21.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2006.0,2.0,24.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44055,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,6.549,160.0,6.8,124.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015.0,1.0,23.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Adding a Constant for Statsmodels

In [ ]:
##import statsmodels correctly


> Tip: make sure that add_constant actually added a new column! You may need to change the parameter `has_constant` to "add"

In [ ]:
## Make final X_train_df and X_test_df with constants added


# Modeling

## Statsmodels OLS

In [ ]:
## instantiate an OLS model WITH the training data.

## Fit the model and view the summary


In [ ]:
## Get train data performance from skearn to confirm matches OLS


## Get test data performance


# The Assumptions of Linear Regression

- The 4 Assumptions of a Linear Regression are:
    - Linearity: That the input features have a linear relationship with the target.
    - Independence of features (AKA Little-to-No Multicollinearity): That the features are not strongly related to other features.
    - **Normality: The model's residuals are approximately normally distributed.**
    - **Homoscedasticity: The model residuals have equal variance across all predictions.**


### QQ-Plot for Checking for Normality

In [ ]:
## Create a Q-QPlot

# first calculate residuals 


## then use sm's qqplot


### Residual Plot for Checking Homoscedasticity

In [ ]:
## Plot scatterplot with y_hat_test vs resids


### Putting it all together

In [ ]:
def evaluate_ols(result,X_train_df, y_train):
    """Plots a Q-Q Plot and residual plot for a statsmodels OLS regression.
    """
    
    ## save residuals from result
    y_pred = result.predict(X_train_df)
    resid = y_train - y_pred
    
    fig, axes = plt.subplots(ncols=2,figsize=(12,5))
    
    ## Normality 
    sm.graphics.qqplot(resid,line='45',fit=True,ax=axes[0]);
    
    ## Homoscedasticity
    ax = axes[1]
    ax.scatter(y_pred, resid, edgecolor='white',lw=1)
    ax.axhline(0,zorder=0)
    ax.set(ylabel='Residuals',xlabel='Predicted Value');
    plt.tight_layout()
    
evaluate_ols(result,X_train_df, y_train)



> Next class: iterating on our model & interpreting coefficients